In [1]:
import torch
from torch import optim
from functools import partial
from torch.utils.data import DataLoader
import sys
import os
import torch.nn as nn

In [2]:
path_to_helper_files = os.path.join('..', 'py_files')

In [3]:
sys.path.append(path_to_helper_files)

In [4]:
import global_variables
import dataset_helper
import nnet_models
import train_utilities

device = global_variables.device;

In [5]:
MAX_LEN = 48
batchSize = 32
hidden_size = 100
bi = True
lr = 1e-2;

In [6]:
en_train_path = '../Data/iwslt-vi-en/train.tok.en'
vi_train_path = '../Data/iwslt-vi-en/train.tok.vi'

en_val_path = '../Data/iwslt-vi-en/dev.tok.en'
vi_val_path = '../Data/iwslt-vi-en/dev.tok.vi'

In [7]:
saved_language_model_dir = os.path.join('..', 'lang_obj')

In [8]:
dataset_dict = {'train': dataset_helper.LanguagePair(source_name = 'vi', target_name='en', 
                                                    source_path = vi_train_path, target_path = en_train_path, 
                                                    lang_obj_path = saved_language_model_dir, max_len = MAX_LEN ), 
               'val': dataset_helper.LanguagePair(source_name = 'vi', target_name='en', 
                                                    source_path = vi_val_path, target_path = en_val_path, 
                                                    lang_obj_path = saved_language_model_dir, max_len = MAX_LEN ) }

In [9]:
dataloader_dict = {'train': DataLoader(dataset_dict['train'], batch_size = batchSize, 
                                    collate_fn = partial(dataset_helper.vocab_collate_func, MAX_LEN=MAX_LEN),
                                shuffle = True, num_workers=0), 
                  'val': DataLoader(dataset_dict['val'], batch_size = batchSize, 
                                    collate_fn = partial(dataset_helper.vocab_collate_func, MAX_LEN=MAX_LEN),
                                shuffle = True, num_workers=0)}

In [10]:
encoder = nnet_models.EncoderRNN(dataset_dict['train'].source_lang_obj.n_words, hidden_size, bi = bi).to(device)

In [11]:
decoder = nnet_models.DecoderRNN(hidden_size, dataset_dict['train'].target_lang_obj.n_words, bi = bi).to(device)     

In [ ]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.NLLLoss()

In [ ]:
enc, dec, loss_hist, acc_hist = train_utilities.train_model(encoder_optimizer, decoder_optimizer, encoder, decoder, criterion,
                                              MAX_LEN, 'normal', dataloader_dict, 
                                              dataset_dict['train'].target_lang_obj, num_epochs = 10)